In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

In [2]:
import magellan as mg
import numpy as np
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


In [3]:
# read in csv files
A= mg.read_csv('tableA1.csv',key='id')
B= mg.read_csv('tableB1.csv',key='id')

In [4]:
# get attribute list
attr_A= list(A.get_attr_names())
attr_B= list(B.get_attr_names())

In [5]:
# partition table into 2 based on whether attributes have NaN value
# return 2 MTables in mg.MTable format
def partitionTable(table,attr):
    if type(table[attr][0]) != str:
        t1= table[~np.isnan(table[attr])]
        t2= table[np.isnan(table[attr])]
    else:
        t1= table[table[attr]!='']
        t2= table[table[attr]=='']        
    t1= mg.MTable(t1,key='id')
    t2= mg.MTable(t2,key='id')
    return t1, t2

In [6]:
# get partitioned table A based on zipcode
# A1 is the table with zipcode
# A2 is the table without zipcode
A1, A2= partitionTable(A,attr_A[5])

In [7]:
# initialize Attribute equivalence blocker
ab = mg.AttrEquivalenceBlocker()

In [8]:
# Attribute equivalence blocking: block A1 and B using zipcode
D1 = ab.block_tables(A1, B, attr_A[5], attr_B[5], 
                     l_output_attrs=['name'], 
                     r_output_attrs=['name'])

In [9]:
# Attribute equivalence blocking: block A2 and B using city
D2 = ab.block_tables(A2, B, attr_A[3], attr_B[3], 
                     l_output_attrs=['name'], 
                     r_output_attrs=['name'])

In [10]:
# make D1 and D2 inherit properties of A 
# in order to use the union function
D1.set_property('ltable', A)
D2.set_property('ltable', A)

In [172]:
# combine blockers via union
#D3= mg.combine_block_outputs_via_union([D1,D2])

In [169]:
# Overlap blocking over input tables
#ob = mg.OverlapBlocker()

In [170]:
# block D1 using name
#D3 = ob.block_candset(D1, 'name', 'name', word_level=True, overlap_size=1)

In [171]:
# block D2 using name
#D4 = ob.block_candset(D2, 'name', 'name', word_level=True, overlap_size=1)

In [ ]:
dbg= mg.debug_blocker(A,B,D1,output_size= 200) 
# Pradap - Instead of A1 it should
# be A

0%                          100%
[#                             ] | ETA[sec]: 1405.817 

In [16]:
type(A2)

magellan.core.mtable.MTable

In [26]:
# Rule-based blocking over input tables
# first get features that can be used
# Pradap --- set zipcode to str !!! as the elements of it qualifies to be 
# int and string.
A2['zipCode'] = A2['zipCode'].astype(str)
feature_table = mg.get_features_for_blocking(A2,B)

In [159]:
#feature_table

In [155]:
# initialize JVM
mg.init_jvm()

True

In [135]:
# Create rule-based blocker
rb = mg.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.8
rb.add_rule(['zipCode_zipCode_lev(ltuple, rtuple) < 0.6'], feature_table)

True

In [ ]:
#E = rb.block_candset(D1)
E = rb.block_tables(A1, B, l_output_attrs=['name'], r_output_attrs=['name'])

In [125]:
len(E)

0

In [ ]:
E